# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = './D22_DATA/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?
* Ans:  


    1. 使用均值編碼準確率較高  
    2. 原因:  
        1. Label encoder只是單純將資料類別進行編號(0, 1,..., n-1)
        2. 均值編碼是將欄位中，相同類別的目標值取平均，這個平均值就代表著欄位中該類別的編碼值，而這個數值並不是單純的編號，而是與目標值有相關聯，因此在預測上才會有較高的準確率。
        
         舉例來說：用"性別欄位"做均值編碼  
             1. 將性別欄位分群(groupby)找類別
             2. 分群後，同時我們要了解每個類別的目標平均值，因此 data.groupby([col])*['Survived']* , 後方要加上目標欄位
             3. 目標欄位在這邊就是 ['Survived']
             4. 以男性類別來看，每筆資料中男性對應存活是不同的，因此取平均值就可以得到男性存活的機率值，同理女性也是
    
    3. 結論:因為均值編碼先以平均值將類別中做有意義的編號，因此預測的準確率也因此提高
        

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
Labencoder = LabelEncoder()
df_lab = copy.deepcopy(df)
df_temp = pd.DataFrame()
for col in df_lab.columns:
    df_temp[col] = Labencoder.fit_transform(df_lab[col])

trainX_lab = df_temp[:train_num]
model = LogisticRegression(solver='lbfgs')
start = time.time()
print('score : %.3f' % cross_val_score(model, trainX_lab, train_Y, cv=5).mean())
print('time  : %.3f sec' % (time.time() - start))

score : 0.779
time  : 0.180 sec


In [4]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for col in df.columns:
    df_mean = data.groupby([col])['Survived'].mean().reset_index()
    df_mean.columns = [col, f'{col}_mean']
    data = pd.merge(data, df_mean, on=col, how='left')
    data = data.drop([col], axis=1)

trainX_mean = data.drop(['Survived'], axis=1)
start = time.time()
print('score : %.3f' % cross_val_score(model, trainX_mean, train_Y, cv=5).mean())
print('time  : %.3f sec' % (time.time() - start))

score : 1.000
time  : 0.074 sec
